In [104]:
import numpy as np
import pandas as pd

from collections import defaultdict
from sklearn.preprocessing import LabelEncoder

In [3]:
# Importing data
test = pd.read_csv('data/test.csv', index_col='id', na_values=['\\N', 'Non Renseigne'])
train = pd.read_csv('data/train.csv', index_col='id', na_values=['\\N', 'Non Renseigne'])

In [28]:
train.dtypes

STRUCTURE PRESCRIPTRICE     object
PLATEFORME                  object
year                         int64
month                        int64
region                      object
ORIENTATION                 object
NATURE_DIFF                 object
age                        float64
tranche_age                 object
situation                   object
adulte_foyer               float64
pers_a_charge                int64
PROF                        object
LOGEMENT                    object
REVENUS                      int64
cat_rev                     object
CHARGES                      int64
cat_charges                 object
CREDITS                      int64
cat_credit                  object
IMPAYES_DEBUT              float64
cat_impayes                 object
RAV_ouverture                int64
cat_RAV_ouverture           object
RAV_UC                      object
cat_RAV_UC                  object
nb_credits                   int64
CRD                         object
gain_mediation      

In [188]:
train.situation.isna().sum()

98

In [200]:
len(train)

4299

In [205]:
train.age.isna().sum()/len(train)

0.40497790183763666

In [209]:
df_general.median().sort_values(ascending=False)

crd_immo          100000.0
crd_rac            25529.5
crd_amort          15209.0
crd_renouv          6929.5
crd_autres          4106.0
year                2017.0
REVENUS             2000.0
IMPAYES_DEBUT       1095.0
CHARGES              978.5
crd_decouvert        900.0
CREDITS              741.0
gain_mediation       542.0
RAV_ouverture        216.0
age                   52.0
month                  7.0
nb_credits             4.0
nb_renouv              2.0
adulte_foyer           2.0
nb_amort               1.0
nb_decouvert           0.0
nb_immo                0.0
nb_rac                 0.0
nb_autres              0.0
pers_a_charge          0.0
dtype: float64

In [189]:
df_general.isna().sum().sort_values(ascending=False)

gain_mediation             7703
crd_decouvert              7567
crd_rac                    7394
crd_autres                 6988
cat_impayes                6675
IMPAYES_DEBUT              6675
crd_immo                   6529
ORIENTATION                3619
age                        3172
tranche_age                3104
crd_renouv                 2218
crd_amort                  1634
PROF                        805
NATURE_DIFF                 331
moy_eco_jour                183
cat_moy_eco_jour            183
RAV_UC                      177
situation                   177
cat_RAV_UC                  177
adulte_foyer                177
LOGEMENT                    158
region                       49
cat_credit                    1
pers_a_charge                 0
PLATEFORME                    0
year                          0
month                         0
nb_decouvert                  0
nb_autres                     0
nb_rac                        0
nb_immo                       0
nb_credi

In [1]:
# import seaborn as sns
# import numpy as np
# import matplotlib.pyplot as plt

# corr = check_df.corr()
# # Generate a mask for the upper triangle
# mask = np.zeros_like(corr, dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True

# # Set up the matplotlib figure
# f, ax = plt.subplots(figsize=(11, 9))

# # Generate a custom diverging colormap
# cmap = sns.diverging_palette(220, 10, as_cmap=True)

# # Draw the heatmap with the mask and correct aspect ratio
# sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0,
#             square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [273]:
def clean(self):
    # Filling NAs
    return self.fillna(self.mean()).fillna('unknown')

In [107]:
def fit_transform(self):  
    # Encoding all the features as int, and saving an encoder by column
    return self.apply(lambda x: self.dict_encoder[x.name].fit_transform(x))

In [108]:
def inverse_transform(self):
    # Retriving value before encoding
    return self.apply(lambda x: self.dict_encoder[x.name].inverse_transform(x))

In [274]:
methods_and_attributes = {
    'clean' : clean,
    'dict_encoder' : defaultdict(LabelEncoder),
    'fit_transform' : fit_transform,
    'inverse_transform' : inverse_transform
}

In [275]:
for ma in methods_and_attributes.keys():
    setattr(pd.DataFrame, ma, methods_and_attributes[ma])